# Stellar Metallicity Profiles

In [85]:
from multiprocessing import Pool
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic

In [86]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse

In [87]:
figure_setup()
settings = Settings()

In [88]:
def read_data(simulation: str, of: str, to: str) -> tuple:
    """
    This method returns data of interest for this analysis.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    pd.DataFrame
        A data frame with the properties.
    """

    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=settings.disc_std_circ,
        disc_min_circ=settings.disc_min_circ,
        cold_disc_delta_circ=settings.cold_disc_delta_circ,
        bulge_max_specific_energy=settings.bulge_max_specific_energy)
    s.add_metal_abundance(of, to)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df["[Fe/H]"] = s.metal_abundance["Fe/H"][is_real_star & is_main_obj]
    df["r_xy"] = s.rho[is_real_star & is_main_obj]
    df["RegTag"] = s.region_tag[is_real_star & is_main_obj]

    return df

In [89]:
def process_data(simulation: int):
    df = read_data(simulation=simulation, of="Fe", to='H')
    median_by_region = []
    for tag in settings.component_tags.values():
        is_region = (df["RegTag"] == tag)
        median, bin_edges, _ = binned_statistic(
            x=df["r_xy"][is_region & np.isfinite(df["[Fe/H]"])],
            values=df["[Fe/H]"][is_region & np.isfinite(df["[Fe/H]"])],
            statistic="median",
            bins=n_bins,
            range=x_range)
        median_by_region.append(median)
    bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
    return bin_centers, median_by_region

In [90]:
def get_data_for_sample(simulations: list):
    data = Pool().map(process_data, simulations)
    df = pd.DataFrame()
    df["BinCenters_kpc"] = data[0][0]
    for i in range(len(simulations)):
        galaxy, _, _, _ = parse(simulations[i])
        for j in range(len(data[0][1])):
            df[f"FeHProfile_Au{galaxy}_{settings.components[j]}"] = \
                data[i][1][j]
    return df

In [91]:
n_bins = 199
x_range = (1, 200)

## Original Run

### [Fe/H] Profiles for Sample

In [92]:
sample = [f"au{i}_or_l4_s127" for i in settings.galaxies]
df = get_data_for_sample(simulations=sample)

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if __name__ == '__main__':


In [94]:
fig = plt.figure(figsize=(7.2, 7.2))
gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

set_axs_configuration(
    xlim=(1, 200), ylim=(-1.5, 1.5),
    xticks=[1, 10, 100], yticks=[-1, -0.5, 0, 0.5, 1],
    xlabel=r"$r_{xy}$ [ckpc]",
    ylabel=r"$\mathrm{median} \left( \mathrm{[Fe/H]} \right)$",
    axs=axs, n_used=len(sample),
    xscale="log", xticklabels=[1, 10, 100])

for idx, ax in enumerate(axs.flat):
    ax.grid(True, ls='-', lw=0.25, c='silver')
    if idx < len(sample):
        galaxy, _, _, _ = parse(sample[idx])
        for tag in settings.component_tags.values():
            ax.plot(
                df["BinCenters_kpc"],
                df[f"FeHProfile_Au{galaxy}_{settings.components[tag]}"],
                lw=1.0, zorder=10,
                c=settings.component_colors[settings.components[tag]],
                label=settings.component_labels[settings.components[tag]])
        ax.text(x=0.95, y=0.95, size=6.0,
                s=r"$\texttt{" + f"au{galaxy}_or_l4".upper() + "}$",
                ha="right", va="top", transform=ax.transAxes)
    else:
        ax.axis("off")

axs[0, 0].legend(loc="upper left", framealpha=0.0, fontsize=4.0)

for ax in axs.flatten():
    ax.xaxis.label.set_size(8.0)
    ax.yaxis.label.set_size(8.0)

fig.savefig(f"../images/metallicity_profiles/originals.pdf")
plt.close(fig)

### Average [Fe/H] Profile

In [95]:
averages = []
for component in settings.components:
    cols = [f"FeHProfile_Au{galaxy}_{component}" \
        for galaxy in settings.galaxies]
    averages.append(df[cols].mean(axis=1))

In [96]:
fig = plt.figure(figsize=(7.4, 2.0))
gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flat:
    ax.tick_params(which='both', direction="in")
    ax.set_xlim(x_range)
    ax.set_xscale("log")
    ax.set_ylim(-1.5, 0.5)
    ax.set_xticks([1, 10, 100])
    ax.set_xticklabels([1, 10, 100])
    ax.set_ylabel(r"$\left< \mathrm{[Fe/H]} \right>$")
    ax.set_xlabel(r"$r_{xy}$ [ckpc]")
    ax.label_outer()

for idx, component in enumerate(settings.components):
    for galaxy in settings.galaxies:
        axs[idx].plot(df["BinCenters_kpc"],
                      df[f"FeHProfile_Au{galaxy}_{component}"],
                      lw=1.0, color="gainsboro")
    axs[idx].plot(df["BinCenters_kpc"], averages[idx],
                  lw=1.5, color=settings.component_colors[component])
    axs[idx].text(x=0.05, y=0.05, size=8.0, ha='left', va='bottom',
                  s=settings.component_labels[settings.components[idx]],
                  transform=axs[idx].transAxes)

fig.savefig(f"../images/metallicity_profiles/originals_average.pdf")
plt.close(fig)

## Reruns

### [Fe/H] Profiles for Sample

In [97]:
sample = [f"au{i}_re_l4_s251" for i in settings.reruns]
df = get_data_for_sample(simulations=sample)

In [ ]:
fig = plt.figure(figsize=(7.2, 7.2))
gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

set_axs_configuration(
    xlim=(1, 200), ylim=(-1.5, 1.5),
    xticks=[1, 10, 100], yticks=[-1, -0.5, 0, 0.5, 1],
    xlabel=r"$r_{xy}$ [ckpc]",
    ylabel=r"$\mathrm{median} \left( \mathrm{[Fe/H]} \right)$",
    axs=axs, n_used=len(sample),
    xscale="log", xticklabels=[1, 10, 100])

for idx, ax in enumerate(axs.flat):
    ax.grid(True, ls='-', lw=0.25, c='silver')
    if idx < len(sample):
        galaxy, _, _, _ = parse(sample[idx])
        for tag in settings.component_tags.values():
            ax.plot(
                df["BinCenters_kpc"],
                df[f"FeHProfile_Au{galaxy}_{settings.components[tag]}"],
                lw=1.0, zorder=10,
                c=settings.component_colors[settings.components[tag]],
                label=settings.component_labels[settings.components[tag]])
        ax.text(x=0.95, y=0.95, size=6.0,
                s=r"$\texttt{" + f"au{galaxy}_or_l4".upper() + "}$",
                ha="right", va="top", transform=ax.transAxes)
    else:
        ax.axis("off")

axs[0, 0].legend(loc="upper left", framealpha=0.0, fontsize=4.0)

for ax in axs.flatten():
    ax.xaxis.label.set_size(8.0)
    ax.yaxis.label.set_size(8.0)

fig.savefig(f"../images/metallicity_profiles/reruns.pdf")
plt.close(fig)